In [1]:
#pip install google-api-python-client

In [2]:
#importar librerias necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import psycopg2

# Paso 1: Extraccion data API

In [3]:
#guardar la llave en un documento .py llamado 'config' y llamar a la variable 'key'
from config import key

In [4]:
#La función se encuentra en https://github.com/googleapis/google-api-python-client/blob/main/docs/start.md
#escogí youtube pero hay mas opciones por revisar en supported APIs

from googleapiclient.discovery import build
youtubeanna = build('youtube', 'v3', developerKey = key)

In [5]:
#hacemos una variable que contenga el channel ID de las cuentas que queremos analizar
#se encuentran en click derecho view source Ctrl+f https://www.youtube.com/channel/ 

channel_ids = ['UCO8gWvBESNnIdZO6xyLLxTA', # autodinamico
               'UCUt99OukA2saWKgAG2Lb9OQ', # fergurquiza
               'UCSPwimlUyP62TPew_rG8vBg', # Jaime Gabaldoni
               'UCm7e2zwpwK5Sm7kzpGPWM6w', # manuelavasquez1
               'UCL5x3r5iEF7cnEgESDN0rsw', # gonzalo conducir
               'UC9EOEaPURNo0o1s0Inua3Yw', # Frankymostro
               'UCyYZafiawKepWzIMhCIRgNA', # Autocosmos México
               'UCgWyfYsBqo3qieiXs1ACbbw', # carwow América Latina
               'UC3FjkCViowUjmYldrssIMAQ' # chekomariscal
              ]

In [6]:
#creamos variables individuales en caso de necesitarlas mas adelante
autodinamico = 'UCO8gWvBESNnIdZO6xyLLxTA'
fer = 'UCUt99OukA2saWKgAG2Lb9OQ'
manuela = 'UCm7e2zwpwK5Sm7kzpGPWM6w'
cheko = 'UC3FjkCViowUjmYldrssIMAQ'
jaime ='UCSPwimlUyP62TPew_rG8vBg'
gonzalo = 'UCL5x3r5iEF7cnEgESDN0rsw'
franky ='UC9EOEaPURNo0o1s0Inua3Yw'
autocosmos = 'UCyYZafiawKepWzIMhCIRgNA'
carwow = 'UCgWyfYsBqo3qieiXs1ACbbw'

In [7]:
#vamos a probar si funciona la API trayendo como JSON el'snippet,contentDetails,statistics' del canal autodinamico
#Para tener una mejor vista del JSON entrar a jsonformatter.curiousconcept.com
#Esto sirve para localizar los datos que queremos extraer y saber cuantos hay
request = youtubeanna.channels().list(part ='snippet,contentDetails,statistics', id= autodinamico)
response = request.execute()
print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'JVGjaUi7aGSqTM7dl5xA5-X3_D0', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'Cf_t3azxlqkMkvlGU_34VZHh_cM', 'id': 'UCO8gWvBESNnIdZO6xyLLxTA', 'snippet': {'title': 'AutoDinámico', 'description': 'AutoDinámico es el canal donde te mantendremos informado de todo lo que tienes que saber de el mundo de los autos. Te ayudamos a decidir que auto comprar o simplemente curiosear que ofrece un auto en comparación a otro. Todo de una manera relajada y enfocado a lo práctico y la vida diaria con el auto.', 'customUrl': '@autodinamico', 'publishedAt': '2014-11-06T07:40:15Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/tBkrua2Ib5vT76ZaFIKgdgS5eEYcd7j1jsqlqjZ5N0meZ8Da8cEOTSwrr9I5Y5VSJH65aqw9dA=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.com/tBkrua2Ib5vT76ZaFIKgdgS5eEYcd7j1jsqlqjZ5N0meZ8Da8cEOTSwrr9I5Y5VSJH65aqw9dA=s240-c-k-c0x00ffffff-no-r

In [8]:
#creamos una funcion que traiga las estadisticas de todos los canales a una lista

def get_channel_stats(youtubeanna, channel_ids):
    all_youtubers = []
    request = youtubeanna.channels().list(
                part ='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute()
    
    for i in range(len(response['items'])):
        data = dict( channel_name = response ['items'][i]['snippet']['title'], 
                     subscribers = response ['items'][i]['statistics']['subscriberCount'], 
                     views = response ['items'][i]['statistics']['viewCount'],
                     video_count = response ['items'][i]['statistics']['videoCount'])
        all_youtubers.append(data)
        
        
    return all_youtubers

In [9]:
#llamamos a la funcion
get_channel_stats(youtubeanna, channel_ids)

[{'channel_name': 'Chekomariscal',
  'subscribers': '74100',
  'views': '1606755',
  'video_count': '212'},
 {'channel_name': 'FERCHO URQUIZA',
  'subscribers': '732000',
  'views': '56977909',
  'video_count': '251'},
 {'channel_name': 'Manuela Vasquez',
  'subscribers': '385000',
  'views': '82296178',
  'video_count': '603'},
 {'channel_name': 'Frankymostro',
  'subscribers': '340000',
  'views': '40903877',
  'video_count': '626'},
 {'channel_name': 'Autocosmos México',
  'subscribers': '542000',
  'views': '165781549',
  'video_count': '1143'},
 {'channel_name': 'carwow América Latina',
  'subscribers': '1030000',
  'views': '209257603',
  'video_count': '741'},
 {'channel_name': 'Gonzalo. Mi Gusto es: ¡Conducir!',
  'subscribers': '462000',
  'views': '109700401',
  'video_count': '1301'},
 {'channel_name': 'AutoDinámico',
  'subscribers': '1410000',
  'views': '438354444',
  'video_count': '1217'},
 {'channel_name': 'Jaime Gabaldoni',
  'subscribers': '275000',
  'views': '99314

In [10]:
#creamos una variable que contenga el llamado de la funcion hecha
channel_statistics = get_channel_stats(youtubeanna, channel_ids)
channel_statistics

[{'channel_name': 'Autocosmos México',
  'subscribers': '542000',
  'views': '165781549',
  'video_count': '1143'},
 {'channel_name': 'Gonzalo. Mi Gusto es: ¡Conducir!',
  'subscribers': '462000',
  'views': '109700401',
  'video_count': '1301'},
 {'channel_name': 'carwow América Latina',
  'subscribers': '1030000',
  'views': '209257603',
  'video_count': '741'},
 {'channel_name': 'Manuela Vasquez',
  'subscribers': '385000',
  'views': '82296178',
  'video_count': '603'},
 {'channel_name': 'Jaime Gabaldoni',
  'subscribers': '275000',
  'views': '99314188',
  'video_count': '1187'},
 {'channel_name': 'Chekomariscal',
  'subscribers': '74100',
  'views': '1606755',
  'video_count': '212'},
 {'channel_name': 'FERCHO URQUIZA',
  'subscribers': '732000',
  'views': '56977909',
  'video_count': '251'},
 {'channel_name': 'Frankymostro',
  'subscribers': '340000',
  'views': '40903877',
  'video_count': '626'},
 {'channel_name': 'AutoDinámico',
  'subscribers': '1410000',
  'views': '438354

In [11]:
#la variable la pasamos a un data frame para poder tener una mejor vista
channels_frame = pd.DataFrame(channel_statistics)
channels_frame

,channel_name,subscribers,views,video_count
0,Autocosmos México,542000,165781549,1143
1,Gonzalo. Mi Gusto es: ¡Conducir!,462000,109700401,1301
2,carwow América Latina,1030000,209257603,741
3,Manuela Vasquez,385000,82296178,603
4,Jaime Gabaldoni,275000,99314188,1187
5,Chekomariscal,74100,1606755,212
6,FERCHO URQUIZA,732000,56977909,251
7,Frankymostro,340000,40903877,626
8,AutoDinámico,1410000,438354444,1217


# Paso 2: Limpieza de data

In [12]:
#revisar el tipo de datos de cada columna
channels_frame.dtypes

channel_name    object
subscribers     object
views           object
video_count     object
dtype: object

In [13]:
#cambiar de tipo de datos segun corresponda
channels_frame['subscribers'] = pd.to_numeric(channels_frame['subscribers'], errors='coerce')
channels_frame['views'] = pd.to_numeric(channels_frame['views'], errors='coerce')
channels_frame['video_count'] = pd.to_numeric(channels_frame['video_count'], errors='coerce')

In [14]:
#revisar nuevamente el tipo de datos de cada columna y confirmar cambio
channels_frame.dtypes

channel_name    object
subscribers      int64
views            int64
video_count      int64
dtype: object

In [15]:
#ordenar el data frame por la columna "subscribers" de mayor a menor
channels_frame= channels_frame.sort_values('subscribers', ascending=False, ignore_index=True)
channels_frame

,channel_name,subscribers,views,video_count
0,AutoDinámico,1410000,438354444,1217
1,carwow América Latina,1030000,209257603,741
2,FERCHO URQUIZA,732000,56977909,251
3,Autocosmos México,542000,165781549,1143
4,Gonzalo. Mi Gusto es: ¡Conducir!,462000,109700401,1301
5,Manuela Vasquez,385000,82296178,603
6,Frankymostro,340000,40903877,626
7,Jaime Gabaldoni,275000,99314188,1187
8,Chekomariscal,74100,1606755,212


In [16]:
#definir una función que agregue comas a los números
def agregar_comas(num):
    if isinstance(num, (int, float)):
        return '{:,}'.format(num)
    else:
        return num

In [17]:
#aplicar la función a las columnas
channels_frame['subscribers'] = channels_frame['subscribers'].apply(agregar_comas)
channels_frame['views'] = channels_frame['views'].apply(agregar_comas)
channels_frame['video_count'] = channels_frame['video_count'].apply(agregar_comas)
channels_frame

,channel_name,subscribers,views,video_count
0,AutoDinámico,"1,410,000","438,354,444","1,217"
1,carwow América Latina,"1,030,000","209,257,603",741
2,FERCHO URQUIZA,"732,000","56,977,909",251
3,Autocosmos México,"542,000","165,781,549","1,143"
4,Gonzalo. Mi Gusto es: ¡Conducir!,"462,000","109,700,401","1,301"
5,Manuela Vasquez,"385,000","82,296,178",603
6,Frankymostro,"340,000","40,903,877",626
7,Jaime Gabaldoni,"275,000","99,314,188","1,187"
8,Chekomariscal,"74,100","1,606,755",212


# Paso 3: Crear la tabla de destino en redshift

In [18]:
#entrar a DBeaver y crear New Database connection
#ingresar Usuario, Contraseña, Host, port y Database
#ir a tu usuario y crear un nuevo script SQL

CREATE TABLE statistics (
channel_name VARCHAR NOT NULL,   
subscribers VARCHAR NOT NULL,
views VARCHAR NOT null,
video_count VARCHAR NOT null,
PRIMARY KEY (channel_name),
UNIQUE (channel_name)
);

# Paso 4: establecer conexion a Redshift

In [21]:
user = 'annarochave_coderhouse'
host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
data_base = 'data-engineer-database'

In [20]:
from config import redshift_pwd

In [22]:
try:
    conn = psycopg2.connect(
        host= host,
        dbname=data_base,
        user=user,
        password=redshift_pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


# Paso 5: Verificar que la tabla exista

In [23]:
cur = conn.cursor()
#ejecutar una query para verificar si existe la tabla que creamos
cur.execute("SELECT * FROM statistics")
# traer los resultados
results = cur.fetchall()
results

[]

# Paso 6: Enviar resultados de tabla a redshift

In [24]:
#llamamos a la funcion dentro de extras que trae valores, tambn lo hace sqlalquemy pero puede haber errores
#nos permite hacer un INSERT multiple con respecto a las columnas del Dataframe
from psycopg2.extras import execute_values

#establecer el cursor hacia la base de datos
cur = conn.cursor()

#ponemos el nombre de la tabla a la cual queremos insertar los datos
table_name = 'statistics'

#ponemos las columnas 
columns = ['channel_name','subscribers', 'views', 'video_count']

#sacamos los resultados que entran a una matriz en numpy y hace un ciclo para insertar los datos
#es una lista de tuplas
#cada tupla es una lista del data frame
values = [tuple(x) for x in channels_frame.to_numpy()]

#aqui inserta los datos separados por comas y le estamos diciendo que es un string
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"

#comenzar la transaccion que necesita un inicio y una autorizacion 
#te pide el cursor, la query y los valores
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")


In [25]:
#nos vamos a DBeaver y checamos con un SELECT * para ver si funciono

In [26]:
# SIEMPRE HAY QUE CERRAR EL CURSOR Y LA CONEXION!!!!
cur.close()
conn.close()